<a href="https://colab.research.google.com/github/bugcaps/workjob/blob/main/3%EB%8B%A8%EA%B3%84_%EC%A3%BC%EC%86%8C%EC%A0%95%EB%B3%B4GEO%EC%A0%95%EB%B3%B4%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0_%EA%B5%AC%EA%B8%80api_%EB%88%84%EB%9D%BD%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B3%B4%EC%99%84_%EA%B2%BD%EC%83%81%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 드라이브  연결
from google.colab import drive
drive.mount('/content/drvie') 

Mounted at /content/drvie


In [ ]:
WORK_PATH ='/content/drvie/MyDrive/apartHome/실거래가/경상도'

In [ ]:
!ls /content/drvie/MyDrive/apartHome/실거래가/경상도

 2020_21_경상도_40_130.csv
'아파트(매매)__실거래가_2020_경상북도.csv'
'아파트(매매)__실거래가_2020_경상남도.csv'
'아파트(매매)__실거래가_2021_경상남도.csv'
'아파트(매매)__실거래가_2021_경상북도.csv'
 경상도아파트주소.csv
 경상도아파트주소_GEO.csv


In [ ]:
import pandas as pd
geoadds = pd.read_csv(WORK_PATH+'/경상도아파트주소_GEO.csv')


## google api key를 조회하여 설정한다.

In [ ]:
import json

# with를 이용해 파일을 연다.
# json 파일은 같은 폴더에 있다고 가정!
GAPI_KEY =''

with open('/content/drvie/MyDrive/apartHome/apikey.txt') as json_file:
    json_data = json.load(json_file)
   
    # 문자열
    # key가 json_string인 문자열 가져오기
    GAPI_KEY = json_data["api_key"]  

In [ ]:
geoadds.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5490 entries, 0 to 5489
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   주소         5490 non-null   object 
 1   locations  5490 non-null   object 
 2   lon        2874 non-null   float64
 3   lat        2874 non-null   float64
dtypes: float64(2), object(2)
memory usage: 171.7+ KB


In [ ]:
geoadds.columns


Index(['주소', 'locations', 'lon', 'lat'], dtype='object')

In [ ]:
#geoadds.drop("Unnamed: 0", axis=1 , inplace=True)
#geoadds.to_csv('/content/drvie/MyDrive/apartHome/실거래가/서울경기아파트주소_GEO.csv', index = False)

In [ ]:
NaNgeoadds =geoadds.query('lat =="NaN"')
display("====값을 못찾은 row수")
display(NaNgeoadds.info())
#NaNgeoadds.set_index('index')
NaN2000add = NaNgeoadds.iloc[:2000]

NaN2000add

'====값을 못찾은 row수'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2616 entries, 157 to 5489
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   주소         2616 non-null   object 
 1   locations  2616 non-null   object 
 2   lon        0 non-null      float64
 3   lat        0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 102.2+ KB


None

,주소,locations,lon,lat
157,경상남도 거제시 옥포동 옥포로18길 26,"('NA', 'NA')",NaN,NaN
236,경상남도 거창군 가조면 수월리 월포2길 20,"('NA', 'NA')",NaN,NaN
274,경상남도 거창군 거창읍 대동리 강양5길 61-3,"('NA', 'NA')",NaN,NaN
293,경상남도 거창군 거창읍 상림리 거열로8길 13,"('NA', 'NA')",NaN,NaN
300,경상남도 거창군 거창읍 상림리 거열로8길 57,"('NA', 'NA')",NaN,NaN
...,...,...,...,...
4424,경상북도 포항북구 득량동 양학로 119,"('NA', 'NA')",NaN,NaN
4425,경상북도 포항북구 득량동 새천년대로 549-10,"('NA', 'NA')",NaN,NaN
4426,경상북도 포항북구 득량동 양학로9번길 29-2,"('NA', 'NA')",NaN,NaN
4427,경상북도 포항북구 득량동 이동로 48-11,"('NA', 'NA')",NaN,NaN


## 구굴 geo api 를 사용하여 부족한 좌표를 추가동록 작업
* 일 api호출 횟수 2500회 무료 임으로  2000회까지만 호출한다. 
  * 2000회 호출하면 10분정도 소요됨.
  * 2500회 이상호출 되면 비용 지불함.

In [ ]:
!pip install  geopandas 
!pip install geopy
#or in Colab: 
!pip install geocoder

구글api를 활용한 2000건단위 데이터조회 하기

NameError: ignored

In [ ]:
import geopandas as gpd

## geo apikey 적용 데이터 조회
location = gpd.tools.geocode(NaN2000add['주소'], provider='googlev3', api_key=GAPI_KEY)

GeocoderTimedOut: ignored

### 좌표정보를 편지한다.

In [ ]:
#동단위월평균 = 전용면적25.groupby(['단지명','시군구','lon','lat','locations'])['금액'].agg(['mean','max','min','count']).sort_values('count', ascending=False).reset_index()

In [ ]:
## 위도, 경도 분리하여 조회하기
from shapely.geometry import Point
#location["locations"] =location.apply(location["geometry"].x,location["geometry"].y)

location["locations"] = location["geometry"].map(lambda point :"(" + str(point.x) +"," + str(point.y) +")" )
location["lon"] = location["geometry"].map(lambda point :point.x )
location["lat"] = location["geometry"].map(lambda point :point.y )

location.drop(["geometry","address"], axis=1 ,inplace=True)
location

,locations,lon,lat
8847,"(126.9472412,37.40723140000001)",126.947241,37.407231
8848,"(126.9330728,37.3973874)",126.933073,37.397387
8849,"(126.9444509,37.3993628)",126.944451,37.399363
8850,"(126.9465756,37.4006387)",126.946576,37.400639
8851,"(126.9430991,37.39839200000001)",126.943099,37.398392
...,...,...,...
13776,"(126.7812692,37.7556465)",126.781269,37.755646
13779,"(127.013414,37.043909)",127.013414,37.043909
13806,"(126.9794356,37.2001448)",126.979436,37.200145
13811,"(127.0587938,37.2098027)",127.058794,37.209803


In [ ]:
## 저장한다.
location.to_csv(WORK_PATH+'/location_2000.csv')

### location csv 화일을 다시 조회하여 원본데이터에 머지 한다.

In [ ]:
locationcsv= pd.read_csv(WORK_PATH+'/location_2000.csv')
locationcsv
locationcsv.rename(columns = {'Unnamed: 0' : 'index'}, inplace = True)
locationcsv.columns
locationcsv.set_index('index')


,locations,lon,lat
index,,,
8847,"(126.9472412,37.40723140000001)",126.947241,37.407231
8848,"(126.9330728,37.3973874)",126.933073,37.397387
8849,"(126.9444509,37.3993628)",126.944451,37.399363
8850,"(126.9465756,37.4006387)",126.946576,37.400639
8851,"(126.9430991,37.39839200000001)",126.943099,37.398392
...,...,...,...
13776,"(126.7812692,37.7556465)",126.781269,37.755647
13779,"(127.013414,37.043909)",127.013414,37.043909
13806,"(126.9794356,37.2001448)",126.979436,37.200145


### 원본데이터 NaN row에 해당 index로 데이터를 update 한다.
   * geoadds 전체 데이터에서  locationcsv에 누락된 좌표정보를 index를 기준으로 join하여 update한다.
   * **set_index('index' ,inplace=True)**   두개의 객체에 index를 설정하여 join해야한다.

In [ ]:
geoadds = pd.read_csv('/content/drvie/MyDrive/apartHome/실거래가/서울경기아파트주소_GEO.csv')
geoadds.set_index('index' ,inplace=True)  

In [ ]:
locationcsv
locationcsv.set_index('index',inplace=True)  

In [ ]:

## 조인하기

geoadds.update(locationcsv)
# joindata =pd.merge(left = locationcsv, right =geoadds , left_index=True, right_index=True,how = "outer",on = "index")
# joindata

In [ ]:
geoadds

,주소,locations,lon,lat
index,,,,
0,서울특별시 강남구 개포동 언주로 103,"(127.0528827, 37.481299500000006)",127.052883,37.481300
1,서울특별시 강남구 개포동 언주로 3,"(127.05343671789691, 37.483371950000006)",127.053437,37.483372
2,서울특별시 강남구 개포동 개포로 307,"(127.05575767360763, 37.4836782)",127.055758,37.483678
3,서울특별시 강남구 개포동 개포로 310,"(127.0570683, 37.4833807)",127.057068,37.483381
4,서울특별시 강남구 개포동 선릉로 7,"(127.0647353, 37.4792567)",127.064735,37.479257
...,...,...,...,...
13814,경기도 화성시 오산동 동탄감배산로 143,"(127.0897249, 37.1958897)",127.089725,37.195890
13815,경기도 화성시 오산동 동탄기흥로 393-20,"(127.0926572, 37.1866501)",127.092657,37.186650
13816,경기도 화성시 장안면 사곡리 밤밭일원길 131-10,"(126.8097868,37.0764502)",126.809787,37.076450


###  3600개에서 1633개로 줄어듬..

In [ ]:
NaNgeoadds =geoadds.query('lat =="NaN"')
NaNgeoadds

,주소,locations,lon,lat
index,,,,


In [ ]:
# update정보를 갱신한다.
geoadds.to_csv('/content/drvie/MyDrive/apartHome/실거래가/서울경기아파트주소_GEO.csv')